In [1]:
import torch

from garage import wrap_experiment
from garage.envs import GymEnv
from garage.experiment.deterministic import set_seed
from garage.sampler import LocalSampler
from garage.torch.algos import TRPO
from garage.torch.algos import VPG

# from garage.torch.policies import GaussianMLPPolicy
from policies.gaussian_mlp_policy import GaussianMLPPolicy

from garage.torch.value_functions import GaussianMLPValueFunction
from garage.trainer import Trainer

@wrap_experiment(log_dir='trpo-without-linesearch')
def trpo_pendulum(ctxt=None, seed=1):
    """Train TRPO with InvertedDoublePendulum-v2 environment.
    Args:
        ctxt (garage.experiment.ExperimentContext): The experiment
            configuration used by Trainer to create the snapshotter.
        seed (int): Used to seed the random number generator to produce
            determinism.
    """
    set_seed(seed)
    env = GymEnv('MountainCarContinuous-v0')

    trainer = Trainer(ctxt)

    policy = GaussianMLPPolicy(env.spec,
                               hidden_sizes=[32, 32],
                               hidden_nonlinearity=torch.tanh,
                               output_nonlinearity=None)

    value_function = GaussianMLPValueFunction(env_spec=env.spec,
                                              hidden_sizes=(32, 32),
                                              hidden_nonlinearity=torch.tanh,
                                              output_nonlinearity=None)

    sampler = LocalSampler(agents=policy,
                           envs=env,
                           max_episode_length=env.spec.max_episode_length)

    algo = TRPO(env_spec=env.spec,
                policy=policy,
                value_function=value_function,
                sampler=sampler,
                discount=0.99,
                center_adv=False)

    trainer.setup(algo, env)
    trainer.train(n_epochs=200, batch_size=1024)

In [2]:
for i in range(5):
    trpo_pendulum(seed=1234)

2022-09-02 00:21:28 | [trpo_pendulum] Logging to trpo-without-linesearch


d:\Anaconda\lib\site-packages\garage\experiment\deterministic.py:36: UserWarning: Enabeling deterministic mode in PyTorch can have a performance impact when using GPU.
  warnings.warn(


2022-09-02 00:21:29 | [trpo_pendulum] Obtaining samples...
2022-09-02 00:21:30 | [trpo_pendulum] epoch #0 | Saving snapshot...
2022-09-02 00:21:30 | [trpo_pendulum] epoch #0 | Saved
2022-09-02 00:21:30 | [trpo_pendulum] epoch #0 | Time 1.42 s
2022-09-02 00:21:30 | [trpo_pendulum] epoch #0 | EpochTime 1.42 s
----------------------------------  ----------
Evaluation/AverageDiscountedReturn   -10.3554
Evaluation/AverageReturn             -96.1168
Evaluation/Iteration                   0
Evaluation/MaxReturn                 -92.5341
Evaluation/MinReturn                 -99.6994
Evaluation/NumEpisodes                 2
Evaluation/StdReturn                   3.58266
Evaluation/TerminationRate             0
TotalEnvSteps                       1998
----------------------------------  ----------
2022-09-02 00:21:31 | [trpo_pendulum] epoch #1 | Saving snapshot...
2022-09-02 00:21:31 | [trpo_pendulum] epoch #1 | Saved
2022-09-02 00:21:31 | [trpo_pendulum] epoch #1 | Time 2.50 s
2022-09-02 00:21:3